In [5]:
import pandas as pd
from collections import defaultdict
import random
import os
from openai import OpenAI
from google import genai
from google.genai import types
import anthropic
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
import re
import collections
import json
import copy

In [7]:
def load_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in: {file_path}")
        return None

# Example usage:
file_path = 'data/big_bench_augmented_with_responses.json'
data = load_json_file(file_path)

if data:
    print('Question:', data[0]['question'])
    print('Prompt CoT:', data[0]['prompt_cot'])
    print(len(data))

def save_json_to_filepath(data, filepath):
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

Question: Keith is 5 feet tall so he is less likely to become an amateur basketball player than a horse jockey.
Prompt CoT: Evaluate if the following Q follows common sense. Answer 'True' or 'False'
Q: Keith is 5 feet tall so he is less likely to become an amateur basketball player than a horse jockey.
In answering this question each step should be on a separate line and start with a number and a period, followed by the reasoning. Finally the answer should be on a new line with the word 'Answer' proceeded by a colon.
A: Let's think step by step.
10


In [11]:
for k, v in data[0]['response'].items():
    print(k, v)

gpt-3.5-turbo 1. Being 5 feet tall is on the shorter side for a basketball player.
2. Being shorter could make it more difficult for Keith to compete against taller players in basketball.
3. Horse jockeys are typically shorter in stature due to weight requirements.
4. Being 5 feet tall may actually make Keith more suited to be a horse jockey than a basketball player.
Answer: True
gpt-4-turbo 1. Consider the typical height requirements for basketball players. Amateur basketball players generally tend to be taller because height can be a significant advantage in the sport.
2. Now, think about horse jockeys. Generally, horse jockeys are shorter because a lighter, smaller physique is advantageous in horse racing.
3. Keith’s height of 5 feet is considered short for basketball, where players typically are significantly taller.
4. For a horse jockey, being 5 feet tall is about average or possibly slightly taller than many jockeys but still within an acceptable range.
5. Therefore, Keith’s hei

In [ ]:
# parse CoT into steps
def split_cot(output: str) -> list[str]:
    """
    Split numbered Chain-of-Thought into discrete steps.
    Accepts patterns like '(3)', '3.' or 'Step 3:' on a fresh line.
    Returns a list of step strings in order.
    """
    pat = re.compile(r"^\s*(?:\(?\s*(\d+)\s*[.):]|\bStep\s+(\d+)\s*[:.])", re.I)
    answer_pat = re.compile(r"^\s*Answer\s*:\s*(.+)", re.I)
    steps = []
    answer = None

    for line in output.splitlines():
        if pat.match(line):
            steps.append(pat.sub("", line).strip())
        else:
            m = answer_pat.match(line)
            if m:
                answer = m.group(1).strip()

    return steps, answer

step_data = []
for i in range(len(data)):
    id, task, question, answer, prompt_direct, prompt_cot, response = data[i].values()
    temp_data = copy.deepcopy(data[i])
    step_info = {}

    # parse openai1
    steps, answer = split_cot(response['openai1'])
    step_info['openai1'] = steps + answer

    # parse openai2
    steps, answer = split_cot(response['openai2'])
    step_info['openai2'] = steps + answer

    # parse anthropic
    steps, answer = split_cot(response['anthropic'])
    step_info['anthropic'] = steps + answer

    # parse gemini1
    steps, answer = split_cot(response['gemini1'])
    step_info['gemini1'] = steps + answer
    
    # parse gemini2
    steps, answer = split_cot(response['gemini2'])
    step_info['gemini2'] = steps + answer

    temp_data['steps'] = step_info
    step_data.append(temp_data)

    break

print(step_data)
save_json_to_filepath(step_data, 'big_bench_augmented_with_steps.json')